# レコメンドシステム

In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import scipy.spatial as sp
import scipy.sparse as sparse
from sklearn.decomposition import NMF, TruncatedSVD

In [2]:
# ジャッカード係数
def get_jaccard_similarity(x, y):
    return 1 - sp.distance.jaccard(x, y)

In [ ]:
def read_data():
    global topN
    topN = 20
    global data
    data  = pd.read_csv('user_data.csv', encoding='utf8')
    data.drop_duplicates(keep='last', inplace=True)
    data.head()

In [ ]:
def rating():
    # 0/1になるため1.0を代入
    data['rating'] = 1.0
    global rating_matrix
    rating_matrix= data.pivot(index='user_id', columns='topic_name', values='rating')
    rating_matrix.fillna(0, inplace=True)
    global topic_list
    topic_list = np.array(rating_matrix.columns)
    user_list = np.array(rating_matrix.index)
    global rating_matrix_ar

    rating_matrix_ar= np.array(rating_matrix)

In [ ]:
def user_already_follow():
    if sys.argv[1:]:
        user_id = sys.argv[1:][0]
    else:
        print("usage: python recommend.py user_id")
        print("sample command: python recommend.py 2")
        sys.exit()

    global already_followed_topic
    already_followed_topic= np.array(data[data['user_id'] == int(user_id)]['topic_name'])
    print("登録済みのテーマ:".join(map(str, already_followed_topic)))

In [ ]:
def user_similarity_rating():
    user_similarity = []
    target_user_row = rating_matrix_ar[0]
    for row in rating_matrix_ar:
        sim = get_jaccard_similarity(target_user_row, row)
        user_similarity.append(sim)

    user_similarity = np.array(user_similarity)
    idx = user_similarity.argsort()[::-1][1:topN+1]
    global selected_user_similarity
    selected_user_similarity= user_similarity[idx]
    global selected_rating
    selected_rating= rating_matrix_ar[idx]

In [ ]:
def average_similarity_score():
    # 平均類似度を計算
    global avg_score
    avg_score = []

    for col_idx in range(selected_rating.shape[1]):
        weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
        similarity_sum =sum(selected_user_similarity[selected_user_similarity > 0])
        avg_score.append(weight_score / similarity_sum)
    avg_score = np.array(avg_score)

In [ ]:
def user_recommend_topic():
    # 平均類似度の高い上位3テーマを表示
    recommend_num = 3
    counter = 0

    print("あなたへのオススメ")
    for recommended_topic in topic_list[avg_score.argsort()[::-1]]:
        if recommended_topic not in already_followed_topic:
            print(recommended_topic)
            counter += 1
        if recommend_num <= counter:
            break

In [ ]:
def run():
    read_data()
    rating()
    user_already_follow()
    user_similarity_rating()
    average_similarity_score()
    user_recommend_topic()

if __name__ == '__main__':
    run()